<a href="https://colab.research.google.com/github/sp7412/colab/blob/master/distilling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

In [27]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

def softmax_sparse_categorical_crossentropy(labels, logits):
  softmaxed = tf.keras.backend.softmax(logits)

  return tf.keras.losses.sparse_categorical_crossentropy(labels, softmaxed)

In [97]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [98]:
ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [99]:
num_training_examples = len(list(ds_train))
ds_train_iter = ds_train.as_numpy_iterator()
ds_train_images, ds_train_labels = zip(*[ds_train_iter.next() for i in range(num_training_examples)])

In [104]:
ds_train_images = list(map(lambda x:tf.reshape(x,(-1,28,28,1)), ds_train_images))

In [105]:
tf.shape(ds_train_images)

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([60000,    28,    28,     1], dtype=int32)>

In [93]:
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [68]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [69]:
control_model = tf.keras.models.Sequential([
  tf.keras.Input([28, 28]), 
  tf.keras.layers.Reshape([28, 28, 1]), 
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(), 
  #tf.keras.layers.Dense(10, activation='softmax')])
  tf.keras.layers.Dense(10, activation=None)])

#control_model.compile('adam', 'sparse_categorical_crossentropy', ['accuracy'])
control_model.compile('adam', softmax_sparse_categorical_crossentropy, ['accuracy'])
control_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_6 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 13, 13, 64)        640       
_________________________________________________________________
batch_normalization_14 (Batc (None, 13, 13, 64)        256       
_________________________________________________________________
dropout_14 (Dropout)         (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 6, 6, 64)          36928     
_________________________________________________________________
batch_normalization_15 (Batc (None, 6, 6, 64)          256       
_________________________________________________________________
dropout_15 (Dropout)         (None, 6, 6, 64)         

In [70]:
control_model.fit(ds_train, epochs=5)

Epoch 1/5
469/469 [==============================] - 2s 4ms/step - loss: 0.9824 - accuracy: 0.7300
Epoch 2/5
469/469 [==============================] - 2s 4ms/step - loss: 0.3814 - accuracy: 0.9057
Epoch 3/5
469/469 [==============================] - 2s 4ms/step - loss: 0.2623 - accuracy: 0.9297
Epoch 4/5
469/469 [==============================] - 2s 4ms/step - loss: 0.2174 - accuracy: 0.9387
Epoch 5/5
469/469 [==============================] - 2s 4ms/step - loss: 0.1917 - accuracy: 0.9440


In [71]:
test_loss, test_acc = control_model.evaluate(ds_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

79/79 [==============================] - 2s 19ms/step - loss: 0.1754 - accuracy: 0.9486
Test Loss: 0.17537686228752136
Test Accuracy: 0.9485999941825867


In [72]:
teacher_model = tf.keras.models.Sequential([
  tf.keras.Input([28, 28]), 
  tf.keras.layers.Reshape([28, 28, 1]), 
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(), 
  #tf.keras.layers.Dense(10, activation='softmax')])
  tf.keras.layers.Dense(10, activation=None)])

#teacher_model.compile('adam', 'sparse_categorical_crossentropy', ['accuracy'])
teacher_model.compile('adam', softmax_sparse_categorical_crossentropy, ['accuracy'])
teacher_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_7 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 13, 13, 64)        640       
_________________________________________________________________
batch_normalization_16 (Batc (None, 13, 13, 64)        256       
_________________________________________________________________
dropout_16 (Dropout)         (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 6, 6, 64)          36928     
_________________________________________________________________
batch_normalization_17 (Batc (None, 6, 6, 64)          256       
_________________________________________________________________
dropout_17 (Dropout)         (None, 6, 6, 64)         

In [73]:
teacher_model.fit(ds_train, epochs=5)

Epoch 1/5
469/469 [==============================] - 2s 4ms/step - loss: 0.3795 - accuracy: 0.8970
Epoch 2/5
469/469 [==============================] - 2s 4ms/step - loss: 0.1182 - accuracy: 0.9662
Epoch 3/5
469/469 [==============================] - 2s 4ms/step - loss: 0.0884 - accuracy: 0.9741
Epoch 4/5
469/469 [==============================] - 2s 4ms/step - loss: 0.0728 - accuracy: 0.9783
Epoch 5/5
469/469 [==============================] - 2s 4ms/step - loss: 0.0628 - accuracy: 0.9813


In [74]:
test_loss, test_acc = teacher_model.evaluate(ds_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

79/79 [==============================] - 0s 2ms/step - loss: 0.0407 - accuracy: 0.9881
Test Loss: 0.040708426386117935
Test Accuracy: 0.988099992275238


In [ ]:
soft_labels = teacher_model.predict(ds_train_images)

In [77]:
temperature = 3
afterwards_temperature = 1

def temperature_softmax(logits):
  soft_logits = tf.keras.backend.exp(logits / temperature)
  return soft_logits / tf.keras.backend.sum(soft_logits, axis=-1, keepdims=True) / afterwards_temperature

def distillation_loss(labels, logits):
  labels = temperature_softmax(labels)
  logits = temperature_softmax(logits)

  return -tf.keras.backend.mean(labels * tf.keras.backend.log(logits))

In [78]:
student_model = tf.keras.models.Sequential([
  tf.keras.Input([28, 28]), 
  tf.keras.layers.Reshape([28, 28, 1]), 
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(), 
  # tf.keras.layers.Dense(10, activation='softmax')])
  tf.keras.layers.Dense(10, activation=None)])

student_model.compile('adam', distillation_loss, ['accuracy'])
student_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_8 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 13, 13, 64)        640       
_________________________________________________________________
batch_normalization_19 (Batc (None, 13, 13, 64)        256       
_________________________________________________________________
dropout_19 (Dropout)         (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 6, 6, 64)          36928     
_________________________________________________________________
batch_normalization_20 (Batc (None, 6, 6, 64)          256       
_________________________________________________________________
dropout_20 (Dropout)         (None, 6, 6, 64)         

In [85]:
ds_train_images[0]

array([[[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        

In [80]:
student_model.fit(ds_train_images, soft_labels, epochs=30)

ValueError: ignored